# AUTOMATIZACION DEL SEGUIMIENTO A POLÍTICAS DE OPERACIÓN

In [ ]:
#Librerias

!pip install fuzzywuzzy==0.18.0
!pip install python-Levenshtein==0.26.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.1 MB/s eta 0:00:00


CONECTAR CON MYDRIVE

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')  # Montar Google Drive

# Revisar si la ruta de la carpeta existe
carpeta_base = "/content/drive/MyDrive/Especializacion"
if os.path.exists(carpeta_base):
    print("✅ La carpeta existe en Google Drive.")
else:
    print("❌ La carpeta no fue encontrada en Google Drive. Revisa si la has montado correctamente.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ La carpeta existe en Google Drive.


In [ ]:
import os

ruta_archivo = "/content/drive/MyDrive/Especializacion/MEDE01.05.02.p025.f006_SEGUIMIENTO_AL_POLÍTICAS_DE_OPERACIÓN.xlsx"

# Verificar si el archivo existe
if os.path.exists(ruta_archivo):
    print("✅ El archivo existe en la ruta especificada.")
else:
    print("❌ El archivo NO se encuentra en la ruta.")

✅ El archivo existe en la ruta especificada.


CREAR LINEAMIENTO DE FUNCIONES PARA INCLUIR DENTRO DE FORMATO ESTABLECIDO POR LA ALCALDÍA SANTIAGO DE CALI

In [ ]:
import traceback
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, Alignment

def formatear_y_guardar_excel(df, ruta_archivo):
    try:
        # Cargar el archivo Excel usando openpyxl
        book = load_workbook(ruta_archivo)
        hoja = book.active

        # Leer el contenido del archivo desde la fila 9 usando pandas
        df_archivo = pd.read_excel(ruta_archivo, engine='openpyxl', skiprows=8)

        # Comprobar que df tenga el mismo número de columnas que df_archivo
        if df.shape[1] != df_archivo.shape[1]:
            raise ValueError("El número de columnas en el DataFrame no coincide con el archivo Excel.")

        # Convertir todos los valores de df y df_archivo a strings para evitar problemas con NaN
        df = df.astype(str).fillna('')
        df_archivo = df_archivo.astype(str).fillna('')

        # Modificar df_archivo a partir de la fila 9 sin tocar las primeras 8 filas
        for idx, col in enumerate(df.columns):
            df_archivo.iloc[:, idx] = df[col].reset_index(drop=True)

        # Escribir los cambios en la hoja de Excel desde la fila 10
        for r in range(len(df_archivo)):
            for c in range(len(df_archivo.columns)):
                valor = df_archivo.iat[r, c]
                hoja.cell(row=r + 10, column=c + 1, value='' if pd.isna(valor) else valor)

        # Eliminar todas las hojas excepto la activa
        for sheet in book.sheetnames:
            if sheet != hoja.title:
                std = book[sheet]
                book.remove(std)

        # Ajustar el alto de las filas basado en el contenido
        for row in hoja.iter_rows(min_row=10, max_row=hoja.max_row):  # Comienza desde la fila 10
            max_height = 0
            for cell in row:
                if cell.value:
                    # Estimar la altura requerida para el contenido
                    cell_height = (len(str(cell.value)) // 20 + 1) * 15  # Ajuste estimado
                    max_height = max(max_height, cell_height)
            hoja.row_dimensions[row[0].row].height = max_height

        # Definir un borde simple
        border = Border(left=Side(style='thin'), right=Side(style='thin'),
                        top=Side(style='thin'), bottom=Side(style='thin'))

        # Recorrer las filas desde la fila 10
        for row in hoja.iter_rows(min_row=10, max_row=hoja.max_row):
            for cell in row:
                # Ajustar alineación: centrar y habilitar ajuste de texto
                cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

        return book

    except Exception as e:
        print(f"Ocurrió un error: {e}")
        traceback.print_exc()
        return None


PROCESO DE EXTRACCIÓN Y LIMPIEZA DE POLÍTICAS DE OPERACIÓN

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

def obtener_politicas_operacion(index):
    url = f'https://sig.cali.gov.co/app.php/staff/document/viewPublic?index={index}'

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Error al obtener el documento. Código de estado: {response.status_code}")
        return None

    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")

    # Expresión regular para detectar números de politicas (Ejemplo: "1- ", "2. ", entre otras.)
    regex_numero = re.compile(r"^\s*(\d+)[.-]\s*")

    politicas = []
    textos_unicos = set()  # Conjunto para evitar duplicados
    consecutivo_politica = 1  # Numeración de políticas

    # 1. Buscar políticas dentro de <ol> <li>
    ol_tag = soup.find("ol")
    if ol_tag:
        for li in ol_tag.find_all("li"):
            text_content = li.get_text(" ", strip=True)
            text_content_clean = regex_numero.sub("", text_content)

            # Evitar duplicados
            if text_content_clean and text_content_clean not in textos_unicos:
                politicas.append([consecutivo_politica, text_content_clean])
                textos_unicos.add(text_content_clean)
                consecutivo_politica += 1

    # 2. Buscar políticas dentro de <p> (incluyendo textos dentro de <span>)
    for p in soup.find_all("p"):
        text_content = " ".join(p.stripped_strings)  # Extrae el texto combinando contenido de <p> y <span>
        match = regex_numero.match(text_content)

        # Evitar duplicados
        if match:
            text_content_clean = regex_numero.sub("", text_content)
            if text_content_clean and text_content_clean not in textos_unicos:
                politicas.append([consecutivo_politica, text_content_clean])
                textos_unicos.add(text_content_clean)
                consecutivo_politica += 1

    # 3. Buscar políticas dentro de <span> (solo si no se capturaron en <p>)
    for span in soup.find_all("span"):
        text_content = span.get_text(" ", strip=True)
        match = regex_numero.match(text_content)

        # Evitar duplicados
        if match:
            text_content_clean = regex_numero.sub("", text_content)
            if text_content_clean and text_content_clean not in textos_unicos:
                politicas.append([consecutivo_politica, text_content_clean])
                textos_unicos.add(text_content_clean)
                consecutivo_politica += 1

    # DataFrame con los resultados
    df = pd.DataFrame(politicas, columns=["No DE POLÍTICA DE OPERACIÓN", "POLÍTICA DE OPERACIÓN"])
    df["REPORTE DEL TRIMESTRE"] = ""  # Agregar la columna faltante con valores NaN

    return df

# Prueba
index = 2778  # Reemplazar con el índice real del documento
df_politicas = obtener_politicas_operacion(index)

if df_politicas is not None:
    from IPython.display import display
    display(df_politicas)  # Mostrar DataFrame en Jupyter Notebook
    df_politicas.to_csv("politicas_operacion.csv", index=False)  # Guardar CSV
    print("Datos guardados en 'politicas_operacion.csv'")


,No DE POLÍTICA DE OPERACIÓN,POLÍTICA DE OPERACIÓN,REPORTE DEL TRIMESTRE
0,1,El Departamento Administrativo de Control Dis...,
1,2,El Departamento Administrativo de Control Disc...,
2,3,El Departamento Administrativo de Control Dis...,
3,4,El Departamento Administrativo de Control Disc...,
4,5,El Departamento Administrativo de Control Dis...,
5,6,El Departamento Administrativo de Control Dis...,


Datos guardados en 'politicas_operacion.csv'


PROCESO DE TRANSFORMACIÓN DE POLITICAS DE OPERACIÓN

In [ ]:
import pandas as pd
import numpy as np
import requests

# URL de la API para obtener procesos (40)
url_procesos = "https://sig.cali.gov.co/api.php/staff/v2/proceso?"

# Token de autenticación
oauth_token = "12ee57a2648f9d5a3732d0fe97930f53"

# Parámetros de la solicitud
params = {'oauth_token': oauth_token}

# Hacer la solicitud GET para obtener los procesos
response = requests.get(url_procesos, params=params)

# Verificar el estado de la respuesta
if response.status_code == 200:
    data = response.json()
    if data['status'] == 'ok':
        # Crear un DataFrame con los datos de procesos
        proceso = pd.DataFrame(data['data'])

        # Seleccionar y renombrar columnas necesarias
        proceso = proceso[['documento(s):nombre', 'documento(s):codigo']]
        proceso = proceso.rename(columns={
            'documento(s):nombre': 'nombre_proceso',
            'documento(s):codigo': 'codigo_proceso'
        })

        # Limpieza de nombres de procesos
        proceso['nombre_proceso'] = proceso['nombre_proceso'].str.replace('MODELACIÓN DE PROCESO ', '', regex=False)
        proceso['nombre_proceso'] = proceso['nombre_proceso'].str.replace('MODELACIÓN DEL PROCESO ', '', regex=False)
        proceso['nombre_proceso'] = proceso['nombre_proceso'].str.replace('MODELACION DE PROCESO ', '', regex=False)
        proceso['nombre_proceso'] = proceso['nombre_proceso'].str.replace('MODELACION DEL PROCESO ', '', regex=False)

        # Eliminar '.MPR001' del código del proceso
        proceso['codigo_proceso'] = proceso['codigo_proceso'].str.replace('.MPR001', '', regex=False)

        # Crear una nueva columna combinada
        proceso['codigo_nombre_proceso'] = proceso['codigo_proceso'] + ' ' + proceso['nombre_proceso']
    else:
        print(f"Error en los datos de procesos: {data['status']}")
else:
    print(f"Error en la solicitud de procesos: {response.status_code}")

# URL de la API para obtener políticas de operación
url_politicas = "https://sig.cali.gov.co/api.php/staff/v2/politicas?"

# Hacer la solicitud GET para obtener las políticas de operación
response = requests.get(url_politicas, params=params)

# Verificar el estado de la respuesta
if response.status_code == 200:
    data = response.json()
    if data['status'] == 'ok':
        # Crear un DataFrame con los datos de políticas de operación
        politicas_operacion = pd.DataFrame(data['data'])

        # Renombrar columnas
        politicas_operacion = politicas_operacion.rename(columns={
            'documento(s):documento_index': 'index_politicas_operacion',
            'documento(s):nombre': 'nombre_politicas_operacion',
            'documento(s):codigo': 'codigo_politicas_operacion'
        })

        # Eliminar frases innecesarias en los nombres de políticas
        politicas_operacion['nombre_politicas_operacion'] = politicas_operacion['nombre_politicas_operacion'].str.replace('POLÍTICA DE OPERACIÓN DEL PROCESO ', '', regex=False)
        politicas_operacion['nombre_politicas_operacion'] = politicas_operacion['nombre_politicas_operacion'].str.replace('POLÍTICA DE OPERACIÓN ', '', regex=False)

        # Extraer código del proceso desde el código de la política de operación
        politicas_operacion['codigo_proceso'] = politicas_operacion['codigo_politicas_operacion'].str[:9]

    else:
        print(f"Error en los datos de políticas: {data['status']}")
else:
    print(f"Error en la solicitud de políticas: {response.status_code}")

# Fusión con los datos de proceso
df = politicas_operacion.merge(proceso, how='left', on='codigo_proceso')

# Verificar y mostrar el DataFrame final
print(df.head())



LOAD - PROCESO DE CARGA DE POLITICAS DE OPERACIÓN

In [ ]:
# INTEGRAR FUNCIONES, ORGANIZAR CARPETAS Y HACER WEB SCRAPPING

import os
import pandas as pd
from openpyxl import Workbook

# Preguntar al usuario por el nombre del trimestre
trimestre = input("Ingrese el nombre del trimestre (por ejemplo, IV_trimestre): ")

# Define la ruta principal en Google Drive
main_folder = f"/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/{trimestre}"
os.makedirs(main_folder, exist_ok=True)

# Definir la carpeta "POLITICAS DE OPERACION"
politicas_de_operacion_folder = os.path.join(main_folder, "POLITICAS DE OPERACION")
os.makedirs(politicas_de_operacion_folder, exist_ok=True)

# Itera sobre las filas del DataFrame de procesos y políticas
for index, row in df.iterrows():
    proceso = str(row['codigo_nombre_proceso'])  # Convertir a cadena

    # Verifica si el proceso es NaN
    if pd.isna(proceso):
        print(f"Fila {index}: Proceso NaN. Saltando...")
        continue  # Salta esta iteración si hay NaN

    # Limpiar nombres
    proceso_limpio = proceso.strip()  # Elimina espacios en blanco

    # Crear la carpeta del proceso dentro de "POLITICAS DE OPERACION"
    folder_path = os.path.join(politicas_de_operacion_folder, proceso_limpio)
    os.makedirs(folder_path, exist_ok=True)

    # Definir el nombre del archivo Excel
    excel_filename = f"{row['codigo_proceso']} {row['nombre_proceso']}.xlsx"
    save_path = os.path.join(folder_path, excel_filename)

    # Obtener el DataFrame de políticas de operación asociado al proceso
    try:
        df_politicas_operacion = obtener_politicas_operacion(row['index_politicas_operacion'])

        # Verificar si df_politicas_operacion contiene datos
        if df_politicas_operacion.empty:
            print(f"DataFrame vacío para {proceso}. Saltando...")
            continue

        book = formatear_y_guardar_excel(df_politicas_operacion, ruta_archivo) # Llama a la función con el nuevo DataFrame y la imagen

        # Cargar y formatear usando la plantilla
        if book:
            # Guardar el archivo Excel en la ruta especificada
            book.save(save_path)

        else:

            print(f"Ocurrió un error al generar el archivo para {proceso}")

        # Crear carpetas para cada número de política dentro de la carpeta del proceso
        for _, politica in df_politicas_operacion.iterrows():
            numero_politica = str(politica['No DE POLÍTICA DE OPERACIÓN']).strip()
            politica_folder = os.path.join(folder_path, f"Política {numero_politica}")
            os.makedirs(politica_folder, exist_ok=True)

    except Exception as e:
        print(f"Error al obtener o guardar datos para {proceso}: {e}")

Ingrese el nombre del trimestre (por ejemplo, IV_trimestre): GYA


PROCESO DE VISUALIZACIÓN DE POLITICAS DE OPERACIÓN

In [ ]:
# CREAR UN SOLO .xlsx para visualización

import pandas as pd

# Lista de archivos Excel con su respectiva ruta
archivos = [
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAGT04.03 GESTION DOCUMENTAL/MAGT04.03 GESTION DOCUMENTAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAGT04.04 ADMINISTRACIÓN DE TECNOLOGÍAS DE LA INFORMACIÓN Y LAS COMUNICACIONES (TIC)/MAGT04.04 ADMINISTRACIÓN DE TECNOLOGÍAS DE LA INFORMACIÓN Y LAS COMUNICACIONES (TIC).xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAGT04.05 ATENCION AL USUARIO/MAGT04.05 ATENCION AL USUARIO.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAHP03.01 GESTIÓN TRIBUTARIA/MAHP03.01 GESTIÓN TRIBUTARIA.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAHP03.02 ADMINISTRACION DE TESORERIA/MAHP03.02 ADMINISTRACION DE TESORERIA.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAHP03.03 CONTABILIDAD GENERAL/MAHP03.03 CONTABILIDAD GENERAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAHP03.06 GESTION DE FINANZAS PUBLICAS/MAHP03.06 GESTION DE FINANZAS PUBLICAS.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAJA01.01 GESTIÓN JURÍDICA/MAJA01.01 GESTIÓN JURÍDICA.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAJA01.03 ADMINISTRACIÓN DE BIENES INMUEBLES, MUEBLES Y AUTOMOTORES/MAJA01.03 ADMINISTRACIÓN DE BIENES INMUEBLES, MUEBLES Y AUTOMOTORES.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MAJA01.04 GESTIÓN CONTRACTUAL/MAJA01.04 GESTIÓN CONTRACTUAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MATH02.06 GESTIÓN Y DESARROLLO HUMANO/MATH02.06 GESTIÓN Y DESARROLLO HUMANO.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MATH02.07 GESTIÓN DE SEGURIDAD SOCIAL INTEGRAL/MATH02.07 GESTIÓN DE SEGURIDAD SOCIAL INTEGRAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MATH02.08 LIQUIDACIONES LABORALES/MATH02.08 LIQUIDACIONES LABORALES.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MCCO01.02 CONTROL INTERNO A LA GESTIÓN/MCCO01.02 CONTROL INTERNO A LA GESTIÓN.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MCCO01.03 CONTROL DISCIPLINARIO/MCCO01.03 CONTROL DISCIPLINARIO.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MEDE01.03 PLANEACIÓN ECONÓMICA Y SOCIAL/MEDE01.03 PLANEACIÓN ECONÓMICA Y SOCIAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MEDE01.04 PLANEACIÓN FÍSICA Y DEL ORDENAMIENTO TERRITORIAL/MEDE01.04 PLANEACIÓN FÍSICA Y DEL ORDENAMIENTO TERRITORIAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MEDE01.05 PLANEACIÓN INSTITUCIONAL/MEDE01.05 PLANEACIÓN INSTITUCIONAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MEDE01.06 COMUNICACIÓN PÚBLICA/MEDE01.06 COMUNICACIÓN PÚBLICA.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MEDE01.07 INFORMACIÓN ESTRATÉGICA/MEDE01.07 INFORMACIÓN ESTRATÉGICA.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MEDE01.08 GESTIÓN DE LA INNOVACIÓN DIGITAL/MEDE01.08 GESTIÓN DE LA INNOVACIÓN DIGITAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MEDE01.09 GESTIÓN DEL BUEN GOBIERNO/MEDE01.09 GESTIÓN DEL BUEN GOBIERNO.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMCS03.01 CONVIVENCIA Y FORTALECIMIENTO SOCIAL/MMCS03.01 CONVIVENCIA Y FORTALECIMIENTO SOCIAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMCS03.02 CONTROL Y MANTENIMIENTO DEL ORDEN PÚBLICO/MMCS03.02 CONTROL Y MANTENIMIENTO DEL ORDEN PÚBLICO.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMCS03.03 GESTIÓN DEL TRÁNSITO Y TRANSPORTE/MMCS03.03 GESTIÓN DEL TRÁNSITO Y TRANSPORTE.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMCS03.05 GESTIÓN DEL RIESGO DE DESASTRES/MMCS03.05 GESTIÓN DEL RIESGO DE DESASTRES.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMCS03.06 GESTIÓN DE PAZ Y CULTURA CIUDADANA/MMCS03.06 GESTIÓN DE PAZ Y CULTURA CIUDADANA.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDI02.01 GESTIÓN CATASTRAL/MMDI02.01 GESTIÓN CATASTRAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDI02.02 DESARROLLO FÍSICO/MMDI02.02 DESARROLLO FÍSICO.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDI02.03 DESARROLLO ECONÓMICO Y COMPETITIVIDAD/MMDI02.03 DESARROLLO ECONÓMICO Y COMPETITIVIDAD.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDI02.04 SUSTENTABILIDAD AMBIENTAL/MMDI02.04 SUSTENTABILIDAD AMBIENTAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDS01.01 PRESTACIÓN DEL SERVICIO EDUCATIVO/MMDS01.01 PRESTACIÓN DEL SERVICIO EDUCATIVO.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDS01.02 SERVICIO DE VIVIENDA SOCIAL/MMDS01.02 SERVICIO DE VIVIENDA SOCIAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDS01.03 SERVICIO DE SALUD PUBLICA/MMDS01.03 SERVICIO DE SALUD PUBLICA.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDS01.04 SERVICIO DE DEPORTE Y RECREACIÓN/MMDS01.04 SERVICIO DE DEPORTE Y RECREACIÓN.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDS01.07 ATENCIÓN A LA COMUNIDAD Y GRUPOS POBLACIONALES/MMDS01.07 ATENCIÓN A LA COMUNIDAD Y GRUPOS POBLACIONALES.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDS01.08 SERVICIOS PÚBLICOS/MMDS01.08 SERVICIOS PÚBLICOS.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDS01.10 GESTIÓN CULTURAL/MMDS01.10 GESTIÓN CULTURAL.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMDS01.11 GESTIÓN DEL TURISMO/MMDS01.11 GESTIÓN DEL TURISMO.xlsx",
    "/content/drive/MyDrive/Seguimientos control operacional/Seguimientos Vigencia 2025/GYA/POLITICAS DE OPERACION/MMPS04.01 PARTICIPACIÓN CIUDADANA Y GESTIÓN COMUNITARIA/MMPS04.01 PARTICIPACIÓN CIUDADANA Y GESTIÓN COMUNITARIA.xlsx",

]

# Lista para almacenar los DataFrames
df_list = []

# Leer archivos de Excel
for archivo in archivos:
    try:
        df = pd.read_excel(archivo)
        if not df.empty:

            df["Fuente"] = os.path.splitext(os.path.basename(archivo))[0]  # Agregar columna con el origen del archivo
            df_list.append(df)
        else:
            print(f"⚠️ Advertencia: El archivo {archivo} está vacío y no será incluido.")
    except Exception as e:
        print(f"❌ Error al leer {archivo}: {e}")

# Verificar si hay DataFrames para concatenar
if df_list:
    df_final = pd.concat(df_list, ignore_index=True)
    # Guardar el archivo unificado
    ruta_salida = "/content/drive/MyDrive/datos_unificados.xlsx"
    df_final.to_excel(ruta_salida, index=False)
    print(f"✅ Archivos combinados y guardados en: {ruta_salida}")
else:
    print("❌ No se encontraron archivos válidos para combinar.")
